In [1]:
"""We translate the over-refusal benchmark XSTest to more languages for multilingual evaluation."""
## remember to add the original english things back too!
## import these into your venv!
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import json
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# hf 
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
login(secret_value_0)

# globals
MODEL_PATH = "facebook/nllb-200-distilled-600M"  # full 12.9B model: CohereLabs/aya-101
DEVICE= "cuda:0"
BATCH_SIZE = 2

# data paths
path_to_xstest = "data/xstest.jsonl"
test_texts = ["Das ist ein Test", "Katzen sind schon cute"] 

iso_target_langs = ["zh", "it", "vi", "ar", "ko", "th", "bn", "sw", "jv"]
nllb_target_langs =  ['zho_Hans', 'ita_Latn', 'vie_Latn', 'arb_Arab', 'kor_Hang', 'tha_Thai', 'ben_Beng', 'swh_Latn', 'jav_Latn'] ## I translate a bunch of stuff to Latin script that aren't intially 8or at all) that script, have to look at that ###

2025-07-10 11:24:54.229543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752146694.408697      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752146694.457433      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def check(x, name_of_x =False):
    """helper function for checking shapes and types during debugging."""
    print("====CHECKING======")
    if name_of_x:
        print(f"name:{name_of_x}")
    print(f"type: {type(x)}")
    if x.shape:
        print(f"shape: {x.shape}")

def translate(texts: list, target_lang: str) -> list:
    """Translate a list of prompts to a target language."""

    # build translation pipe specifically for that language.
    translator = pipeline(
        "translation",
        model=MODEL_PATH,
        src_lang="eng_Latn",
        tgt_lang=target_lang,
        batch_size=BATCH_SIZE,
        max_length=512,
        device=DEVICE
    )
    
    translated_texts = translator(
        texts
    ) 

    translated = translator(texts)
    return [t["translation_text"] for t in translated]

### Load in Data

In [3]:
from datasets import load_dataset
import pandas as pd

hf_dataset = load_dataset("bench-llm/or-bench", "or-bench-80k")
df = pd.DataFrame(hf_dataset['train']) # 80k samples

,prompt,category
0,What are some strategies to promote a respectf...,deception
1,How can one encourage community building and s...,deception
2,What are some effective communication strategi...,deception
3,Research techniques for sharing thought-provok...,deception
4,What are some tactics to subtly influence a pe...,deception
5,Can you guide me in creating a site that featu...,deception
6,Can you suggest ways to maintain independence ...,deception
7,Understand ways to ensure a safe and respectfu...,deception
8,Explore various communication strategies to su...,deception
9,Can you explain how to recognize and respond t...,deception


In [4]:
df.head(10)

,prompt,category
0,What are some strategies to promote a respectf...,deception
1,How can one encourage community building and s...,deception
2,What are some effective communication strategi...,deception
3,Research techniques for sharing thought-provok...,deception
4,What are some tactics to subtly influence a pe...,deception
5,Can you guide me in creating a site that featu...,deception
6,Can you suggest ways to maintain independence ...,deception
7,Understand ways to ensure a safe and respectfu...,deception
8,Explore various communication strategies to su...,deception
9,Can you explain how to recognize and respond t...,deception


In [16]:
# sampling.
# we sample an equal percentage from all categories to make sure we get a representative sample
SAMPLE_SIZE = 200
nrows = len(df)

sampled_df = df.groupby('category').\
    apply(lambda x: x.sample(int((x.count()/nrows)*SAMPLE_SIZE)))

/tmp/ipykernel_36/623358433.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  apply(lambda x: x.sample(int((x.count()/nrows)*SAMPLE_SIZE)))
/tmp/ipykernel_36/623358433.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  apply(lambda x: x.sample(int((x.count()/nrows)*SAMPLE_SIZE)))
/tmp/ipykernel_36/623358433.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  apply(lambda x: x.sample(int((x.count()/nrows)*SAMPLE_SIZE)))
/tmp/ipykernel_36/623358433.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  apply(lambda x: x.sample(int((x.count()/nrows)*SAMPLE_SIZE)))
/tmp/ipykernel_36/623358433.py:7: FutureWarning: Cal

In [24]:
sampled_df.to_csv(f"/kaggle/working/sampled_or_bench_{SAMPLE_SIZE}_prompts.csv", index = True, )

In [30]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Language grouping as required by the model
GROUP2LANG = {
    1: ["da", "nl", "de", "is", "no", "sv", "af"],
    2: ["ca", "ro", "gl", "it", "pt", "es"],
    3: ["bg", "mk", "sr", "uk", "ru"],
    4: ["id", "ms", "th", "vi", "mg", "fr"],
    5: ["hu", "el", "cs", "pl", "lt", "lv"],
    6: ["ka", "zh", "ja", "ko", "fi", "et"],
    7: ["gu", "hi", "mr", "ne", "ur"],
    8: ["az", "kk", "ky", "tr", "uz", "ar", "he", "fa"],
}

# maps ISO lang codes to full language names for prompt.
ISO_TO_NAME = {
    "ka": "Georgian",
    "zh": "Chinese",
    "ja": "Japanese",
    "ko": "Korean",
    "fi": "Finnish",
    "et": "Estonian",
    "en": "English",
    "fr": "French",
    "de": "German",
    "es": "Spanish",
    "it": "Italian",
    "pt": "Portuguese"
}


LANG2GROUP = {lang: str(group) for group, langs in GROUP2LANG.items() for lang in langs}


def load_model_for_lang(lang_code):
    """Load model and tokenizer based on language group."""
    group_id = LANG2GROUP[lang_code]   # .get(lang_code) 
    
    if group_id is None:
        raise ValueError(f"Language '{lang_code}' not supported.")

    model_name = f"haoranxu/X-ALMA-13B-Group{group_id}"
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    return model, tokenizer

def translate(text, source_lang, target_lang):
    """Translate text from source_lang to target_lang using X-ALMA model."""
    model, tokenizer = load_model_for_lang(target_lang) ## here is the issue...

    # get full language names
    src_name = ISO_TO_NAME.get(source_lang, source_lang)
    tgt_name = ISO_TO_NAME.get(target_lang, target_lang)

    # format our prompt
    prompt = f"Translate this from {src_name} to {tgt_name}:\n{src_name}: {text}\n{tgt_name}:"
    chat_prompt = [{"role": "user", "content": prompt}]
    prompt = tokenizer.apply_chat_template(chat_prompt, tokenize=False, add_generation_prompt=True)

    input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=256, truncation=True).input_ids.cuda()

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            num_beams=5,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
        outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    translation = outputs[0].strip()
    return translation


# Example usage: Translate Chinese text to English
if __name__ == "__main__":
    input_text = "我爱机器翻译。"
    result = translate(input_text, source_lang="zh", target_lang="en")
    print(result)


KeyError: 'en'

In [31]:
def translate_dataframe(sampled_df, prompt_column, target_langs):
    """
    Translates each prompt in the DataFrame into multiple target languages.
    
    Args:
        sampled_df (pd.DataFrame): Input DataFrame with prompts.
        prompt_column (str): Name of the column containing prompts.
        target_langs (List[str]): List of ISO codes to translate into.

    Returns:
        pd.DataFrame: DataFrame with translated samples, ids, and language codes.
    """
    translations = []

    for i, row in sampled_df.iterrows():
        original_text = row[prompt_column]

        for lang in target_langs:
            try:
                translated_text = translate(original_text, source_lang="en", target_lang=lang)
                translations.append({
                    "id": f"{i}_{lang}",
                    "lang": lang,
                    "text": translated_text,
                })
            except Exception as e:
                print(f"Error translating row {i} to {lang}: {e}")
                continue

    return pd.DataFrame(translations)


In [32]:
target_languages = ["zh", "ja", "ko"]
translated_df = translate_dataframe(sampled_df, prompt_column="prompt", target_langs=target_languages)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 